In [3]:
pip install --upgrade streamlit

Note: you may need to restart the kernel to use updated packages.


In [1]:
import streamlit as st
import pandas as pd
import plotly.express as px

In [3]:
df.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,balance_delta_org,balance_delta_dest,...,type_TRANSFER,ml_fraud_score,ml_anomaly_flag,rule_score,rule_reasons,rule_score_norm,llm_score,total_fraud_score,flagged_fraud,llm_explanation
0,278,330218.42,20866.00,351084.42,452419.57,122201.15,0,0,-330218.42,-330218.42,...,False,0.214322,0,40,Balance mismatch detected;,0.40,0,0.208593,0,Not investigated (low risk)
1,15,11647.08,30370.00,18722.92,0.00,0.00,0,0,11647.08,0.00,...,False,0.007399,0,0,NaN,0.00,0,0.004439,0,Not investigated (low risk)
2,10,152264.21,106589.00,258853.21,201303.01,49038.80,0,0,-152264.21,-152264.21,...,False,0.124148,0,40,Balance mismatch detected;,0.40,0,0.154489,0,Not investigated (low risk)
3,403,1551760.63,0.00,0.00,3198359.45,4750120.08,0,0,0.00,1551760.63,...,True,0.430251,0,55,Balance mismatch detected; Transaction type is...,0.55,0,0.368150,0,Not investigated (low risk)
4,206,78172.30,2921331.58,2999503.88,415821.90,337649.60,0,0,-78172.30,-78172.30,...,False,0.226824,0,40,Balance mismatch detected;,0.40,0,0.216094,0,Not investigated (low risk)


In [4]:
df.columns

Index(['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest',
       'newbalanceDest', 'isFraud', 'isFlaggedFraud', 'balance_delta_org',
       'balance_delta_dest', 'hour', 'type_CASH_OUT', 'type_DEBIT',
       'type_PAYMENT', 'type_TRANSFER', 'ml_fraud_score', 'ml_anomaly_flag',
       'rule_score', 'rule_reasons', 'rule_score_norm', 'llm_score',
       'total_fraud_score', 'flagged_fraud', 'llm_explanation'],
      dtype='object')

In [5]:
# Load your processed dataset
@st.cache_data
def load_data():
    df = pd.read_csv("paysim_processed_with_scores.csv")
    return df

df = load_data()

st.set_page_config(page_title="FraudGuard Dashboard", layout="wide")

st.title("💡 FraudGuard — AI Fraud Investigation Dashboard")

# -------------------------
# Sidebar Filters
# -------------------------
st.sidebar.header("Filters")

transaction_id = st.sidebar.text_input("Search by Transaction Step")
sender_filter = st.sidebar.text_input("Sender Account Filter")
receiver_filter = st.sidebar.text_input("Receiver Account Filter")
transaction_type = st.sidebar.multiselect(
    "Transaction Type", options=["CASH_OUT", "TRANSFER", "OTHER"]
)
risk_level = st.sidebar.multiselect(
    "Risk Level", options=["Low", "Medium", "High"]
)
ml_score_range = st.sidebar.slider("ML Fraud Score Range", 0.0, 1.0, (0.0, 1.0))
rule_score_range = st.sidebar.slider("Rule Score Range", 0, 100, (0, 100))
total_score_range = st.sidebar.slider("Total Fraud Score Range", 0.0, 1.0, (0.0, 1.0))
flagged_only = st.sidebar.checkbox("Show Flagged Fraud Only", value=True)

# -------------------------
# Apply filters
# -------------------------
filtered_df = df.copy()

if transaction_id:
    filtered_df = filtered_df[filtered_df["step"].astype(str).str.contains(transaction_id)]
if sender_filter:
    filtered_df = filtered_df[filtered_df["oldbalanceOrg"].astype(str).str.contains(sender_filter)]
if receiver_filter:
    filtered_df = filtered_df[filtered_df["newbalanceDest"].astype(str).str.contains(receiver_filter)]
if transaction_type:
    def map_type(row):
        if row["type_TRANSFER"]: return "TRANSFER"
        elif row["type_CASH_OUT"]: return "CASH_OUT"
        else: return "OTHER"
    filtered_df["type_mapped"] = filtered_df.apply(map_type, axis=1)
    filtered_df = filtered_df[filtered_df["type_mapped"].isin(transaction_type)]
if risk_level:
    filtered_df = filtered_df[filtered_df["risk_level"].isin(risk_level)]
filtered_df = filtered_df[
    (filtered_df["ml_fraud_score"] >= ml_score_range[0]) & 
    (filtered_df["ml_fraud_score"] <= ml_score_range[1])
]
filtered_df = filtered_df[
    (filtered_df["rule_score"] >= rule_score_range[0]) & 
    (filtered_df["rule_score"] <= rule_score_range[1])
]
filtered_df = filtered_df[
    (filtered_df["total_fraud_score"] >= total_score_range[0]) & 
    (filtered_df["total_fraud_score"] <= total_score_range[1])
]
if flagged_only:
    filtered_df = filtered_df[filtered_df["flagged_fraud"] == 1]

# -------------------------
# KPIs / Metrics
# -------------------------
st.subheader("📊 Overview Metrics")
col1, col2, col3, col4, col5 = st.columns(5)
col1.metric("Total Transactions", len(filtered_df))
col2.metric("Flagged Transactions", filtered_df["flagged_fraud"].sum())
col3.metric("Average ML Score", round(filtered_df["ml_fraud_score"].mean(), 2))
col4.metric("Average Rule Score", round(filtered_df["rule_score"].mean(), 2))
col5.metric("Average Total Score", round(filtered_df["total_fraud_score"].mean(), 2))

# -------------------------
# Transaction Table
# -------------------------
st.subheader("💳 Transactions Table")
st.dataframe(
    filtered_df[
        ["step", "amount", "type_TRANSFER", "type_CASH_OUT", "ml_fraud_score",
         "rule_score", "total_fraud_score", "flagged_fraud"]
    ],
    use_container_width=True
)

# -------------------------
# Show LLM Explanation for selected transaction
# -------------------------
st.subheader("📝 LLM Explanation")

if not selected_row.empty:
    selected_step = selected_row.iloc[0]["step"]
    explanation_row = df[df["step"] == selected_step]
    if not explanation_row.empty:
        st.markdown(f"**Transaction Step:** {selected_step}")
        #st.markdown(f"**Risk Level:** {explanation_row.iloc[0]['risk_level']}")
        st.markdown(f"**Key Risk:** {explanation_row.iloc[0]['llm_key_risk']}")
        st.markdown(f"**Recommended Action:** {explanation_row.iloc[0]['llm_recommended_action']}")
    else:
        st.write("No explanation found for this transaction.")
else:
    st.write("Select a transaction from the table above to see its LLM explanation.")

# -------------------------
# Fraud Trend Chart
# -------------------------
st.subheader("📈 Fraud Trend by Hour")
if "hour" in filtered_df.columns:
    fraud_hour = filtered_df.groupby("hour")["flagged_fraud"].sum().reset_index()
    fig = px.bar(fraud_hour, x="hour", y="flagged_fraud", labels={"flagged_fraud": "Flagged Transactions", "hour": "Hour of Day"})
    st.plotly_chart(fig, use_container_width=True)


2026-02-04 18:08:32.218 No runtime found, using MemoryCacheStorageManager
2026-02-04 18:08:32.299 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:08:32.302 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:08:32.302 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:08:32.304 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:08:32.305 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:08:32.306 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:08:32.306 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:08:32.307 Thread 'MainThread':

2026-02-04 18:08:32.422 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:08:32.422 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:08:32.423 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:08:32.424 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:08:32.425 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:08:32.426 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:08:32.428 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:08:32.429 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

NameError: name 'selected_row' is not defined

In [7]:
import streamlit as st
import pandas as pd
import plotly.express as px

# -------------------------
# Page config (MUST be first Streamlit command)
# -------------------------
st.set_page_config(page_title="FraudGuard Dashboard", layout="wide")

# -------------------------
# Load data
# -------------------------
@st.cache_data
def load_data():
    return pd.read_csv("paysim_processed_with_scores.csv")

df = load_data()

st.title("💡 FraudGuard — AI Fraud Investigation Dashboard")

# -------------------------
# Sidebar Filters
# -------------------------
st.sidebar.header("Filters")

transaction_id = st.sidebar.text_input("Search by Transaction Step")
sender_filter = st.sidebar.text_input("Sender Balance Filter")
receiver_filter = st.sidebar.text_input("Receiver Balance Filter")

transaction_type = st.sidebar.multiselect(
    "Transaction Type", options=["CASH_OUT", "TRANSFER", "OTHER"]
)


ml_score_range = st.sidebar.slider("ML Fraud Score Range", 0.0, 1.0, (0.0, 1.0))
rule_score_range = st.sidebar.slider("Rule Score Range", 0, 100, (0, 100))
total_score_range = st.sidebar.slider("Total Fraud Score Range", 0.0, 1.0, (0.0, 1.0))
flagged_only = st.sidebar.checkbox("Show Flagged Fraud Only", value=True)

# -------------------------
# Apply filters
# -------------------------
filtered_df = df.copy()

if transaction_id:
    filtered_df = filtered_df[
        filtered_df["step"].astype(str).str.contains(transaction_id)
    ]

if sender_filter:
    filtered_df = filtered_df[
        filtered_df["oldbalanceOrg"].astype(str).str.contains(sender_filter)
    ]

if receiver_filter:
    filtered_df = filtered_df[
        filtered_df["newbalanceDest"].astype(str).str.contains(receiver_filter)
    ]

if transaction_type:
    def map_type(row):
        if row.get("type_TRANSFER", False):
            return "TRANSFER"
        elif row.get("type_CASH_OUT", False):
            return "CASH_OUT"
        else:
            return "OTHER"

    filtered_df["type_mapped"] = filtered_df.apply(map_type, axis=1)
    filtered_df = filtered_df[filtered_df["type_mapped"].isin(transaction_type)]


filtered_df = filtered_df[
    (filtered_df["ml_fraud_score"].between(*ml_score_range)) &
    (filtered_df["rule_score"].between(*rule_score_range)) &
    (filtered_df["total_fraud_score"].between(*total_score_range))
]

if flagged_only:
    filtered_df = filtered_df[filtered_df["flagged_fraud"] == 1]

# -------------------------
# KPIs
# -------------------------
st.subheader("📊 Overview Metrics")
c1, c2, c3, c4, c5 = st.columns(5)

c1.metric("Total Transactions", len(filtered_df))
c2.metric("Flagged Transactions", int(filtered_df["flagged_fraud"].sum()))
c3.metric("Avg ML Score", round(filtered_df["ml_fraud_score"].mean(), 2))
c4.metric("Avg Rule Score", round(filtered_df["rule_score"].mean(), 2))
c5.metric("Avg Total Score", round(filtered_df["total_fraud_score"].mean(), 2))

# -------------------------
# Transactions Table (WITH SELECTION)
# -------------------------
st.subheader("💳 Transactions Table")

event = st.dataframe(
    filtered_df[
        [
            "step", "amount", "ml_fraud_score",
            "rule_score", "total_fraud_score",
            "flagged_fraud"
        ]
    ],
    use_container_width=True,
    selection_mode="single-row",
    on_select="rerun"
)

# Create selected_row safely
if event.selection.rows:
    selected_row = filtered_df.iloc[event.selection.rows]
else:
    selected_row = pd.DataFrame()

# -------------------------
# LLM Explanation Panel
# -------------------------
st.subheader("📝 LLM Explanation")

if not selected_row.empty:
    selected_step = selected_row.iloc[0]["step"]
    explanation_row = df[df["step"] == selected_step]

    st.markdown(f"**Transaction Step:** `{selected_step}`")
    st.markdown(f"**ML Fraud Score:** {explanation_row.iloc[0]['ml_fraud_score']:.2f}")
    st.markdown(f"**Rule Score:** {explanation_row.iloc[0]['rule_score']}")
    st.markdown(f"**Total Score:** {explanation_row.iloc[0]['total_fraud_score']:.2f}")
    st.markdown("---")
    st.markdown(f"**🧠 LLM Explanation:**")
    st.info(explanation_row.iloc[0]["llm_explanation"])
else:
    st.info("👆 Select a transaction row above to see the LLM explanation.")

# -------------------------
# Fraud Trend Chart
# -------------------------
st.subheader("📈 Fraud Trend by Hour")

if "hour" in filtered_df.columns:
    fraud_hour = (
        filtered_df.groupby("hour")["flagged_fraud"]
        .sum()
        .reset_index()
    )

    fig = px.bar(
        fraud_hour,
        x="hour",
        y="flagged_fraud",
        labels={
            "hour": "Hour of Day",
            "flagged_fraud": "Flagged Transactions"
        }
    )

    st.plotly_chart(fig, use_container_width=True)


2026-02-04 18:21:09.131 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:21:09.132 No runtime found, using MemoryCacheStorageManager
2026-02-04 18:21:09.155 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:21:09.155 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:21:09.157 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:21:09.158 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:21:09.158 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:21:09.159 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:21:09.160 Thread 'MainThread':

2026-02-04 18:21:09.249 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:21:09.250 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:21:09.251 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:21:09.251 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:21:09.252 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:21:09.253 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:21:09.253 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-04 18:21:09.254 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar